# Data section

## Imports

In [1]:
import pandas as pd
import requests as rq
from sklearn.metrics import classification_report

## Data Loading

In [2]:
# Load training and test sets

df_train = pd.read_json('datas/training_set.json')
df_test = pd.read_json('datas/testing_set.json')

print(f"Train shape : {df_train.shape}")
print(f"Test shape : {df_test.shape}")

Train shape : (6035, 2)
Test shape : (1065, 2)


## Info and visualisations of datasets

In [3]:
# Stats on the training set
df_train.describe()

,intent,sentence
count,6035,6035
unique,8,6035
top,irrelevant,C'est censé durer longtemps ? Tu sais combien ...
freq,3852,1


In [4]:
# Informations about colums
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6035 entries, 0 to 6034
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   intent    6035 non-null   object
 1   sentence  6035 non-null   object
dtypes: object(2)
memory usage: 94.4+ KB


In [5]:
# Show 10 first elements
df_train.head(n=10)

,intent,sentence
0,irrelevant,"850€ maximum pour le loyer, à partir de janvie..."
1,irrelevant,D'imprimer
2,purchase,Le meilleur cabriolet hybrid moins de 5m10 min...
3,find-hotel,en ce moment je cher un location pour les vaca...
4,irrelevant,c'est possible de t'utiliser la nuit ?
5,irrelevant,J'ai besoin d'acheter un fusil
6,irrelevant,Vous pouvez réserver pour 09h oui
7,irrelevant,Du 20 au 22 novembre pour 100-150 euros la nuit
8,purchase,Mon docteur m'a suggéré de porter des bandes p...
9,purchase,Commande à effectuer : 30 bloc note petits car...


In [6]:
# Show the 8 different intents
df_train["intent"].value_counts()

irrelevant           3852
purchase              613
find-restaurant       469
find-around-me        383
find-hotel            316
find-train            143
find-flight           142
provide-showtimes     117
Name: intent, dtype: int64

# Model section

## Split datasets in inputs and labels

In [7]:
df_x_train = df_train['sentence']
df_y_train = df_train['intent']
df_x_test = df_test['sentence']
df_y_test = df_test['intent']

print(f"Train data shape : {df_x_train.shape}")
print(f"Train labels shape : {df_y_train.shape}")
print(f"Test data shape : {df_x_test.shape}")
print(f"Test labels shape : {df_y_test.shape}")

Train data shape : (6035,)
Train labels shape : (6035,)
Test data shape : (1065,)
Test labels shape : (1065,)


## Get model predictions for both datasets

In [8]:
route = 'http://localhost:8080/api/intent?'

# Function to get the model's predictions for a given dataset
def predict(datas):
    
    # List of predicted intents
    predicted_labels = []
    # List of probabilities for the predicted intents
    prediction_probabilities = []
    
    # Request the model for each data
    for data in datas:
        
        res = rq.get(route, {'sentence':data}).json()
        predicted_class = max(res, key=res.get)
        predicted_value = max(res.values())
 
        predicted_labels.append(predicted_class)
        prediction_probabilities.append(predicted_value)
        
    return predicted_labels, prediction_probabilities   

In [9]:
# Get both datasets predictions from the model
train_predicted_labels, train_predicted_probabilities = predict(df_x_train)
test_predicted_labels, test_predicted_probabilities = predict(df_x_test)

assert len(train_predicted_labels) == df_x_train.shape[0]
assert len(train_predicted_labels) == df_x_train.shape[0]
assert len(test_predicted_labels) == df_x_test.shape[0]
assert len(test_predicted_labels) == df_x_test.shape[0]

## Compute model's various scores

In [10]:
# Training scores
print(classification_report(df_y_train, train_predicted_labels))

                   precision    recall  f1-score   support

   find-around-me       0.70      0.39      0.50       383
      find-flight       0.96      0.38      0.55       142
       find-hotel       0.86      0.48      0.62       316
  find-restaurant       0.91      0.61      0.73       469
       find-train       0.95      0.51      0.66       143
       irrelevant       0.80      0.98      0.88      3852
provide-showtimes       0.97      0.29      0.45       117
         purchase       0.76      0.56      0.65       613

         accuracy                           0.81      6035
        macro avg       0.86      0.53      0.63      6035
     weighted avg       0.81      0.81      0.79      6035



In [11]:
# Test scores
print(classification_report(df_y_test, test_predicted_labels))

                   precision    recall  f1-score   support

   find-around-me       0.88      0.43      0.58        67
      find-flight       0.88      0.29      0.44        24
       find-hotel       0.78      0.38      0.51        55
  find-restaurant       0.98      0.56      0.71        93
       find-train       0.93      0.67      0.78        21
       irrelevant       0.79      0.98      0.87       677
provide-showtimes       0.80      0.29      0.42        14
         purchase       0.79      0.59      0.67       114

         accuracy                           0.80      1065
        macro avg       0.85      0.52      0.62      1065
     weighted avg       0.82      0.80      0.78      1065

